# Packages

In [ ]:
using LinearAlgebra
using DifferentialEquations, DiffEqCallbacks
using COVID19SchoolReopening
using ProgressMeter
using PyPlot, LaTeXStrings

const CSR = COVID19SchoolReopening

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["axes.grid"] = true
rcParams["font.size"] = 12

# Model Parameters

In [ ]:
# length of simulation, translated from months to days (model time scale)
months = 6
days = months * CSR.DAYS_PER_MONTH
weeks = Int(round(Int, days / 7))
tspan = (0, days*1.0);

In [ ]:
# assuming model time in days

# cohort interaction, between 0 and 1
α = 0.05 * ones(3, 3)
α[1,1] = 1      # cohort 1
α[2,2] = 1      # cohort 2
α[3,3] = 1      # cohort 3

# latent period, time between exposure time and beginning of infectious period
σ = zeros(2)
σ[1] = 1/(3)     # children, 3 days
σ[2] = 1/(3)     # adults, 3 days

# removal rate, 'recovery' from infectious state to non-infectious
γ = zeros(2)
γ[1] = 1/(4)    # child recovery rate, 4 days
γ[2] = 1/(4)    # adult recovery rate, 4 days

# baseline transmission rates
β0 = 1.2
F = [
    0.1     0.25    # child to j
    0.15    0.5     # adult to j
]
β = β0 * F

# population: [children, adults]
demographics = [22., 78.]
total_infected = 2.0
infected = [10., 90.]

params = (α, σ, γ);

In [ ]:
# helper functions to make some of the later code easier to read
function get_compartment(u, i, j)
    c = zero(eltype(u))

    # iterate over cohorts
    for k in 1:size(u, 3)
        c += max(0, u[i,j,k])
    end
    return c
end

infected_child_fraction(u) = get_compartment(u, 3, 1)
removed_child_fraction(u) = get_compartment(u, 4, 1)
cumulative_child_fraction(u) = infected_child_fraction(u) + removed_child_fraction(u)

infected_adult_fraction(u) = get_compartment(u, 3, 2)
removed_adult_fraction(u) = get_compartment(u, 4, 2)
cumulative_adult_fraction(u) = infected_adult_fraction(u) + removed_adult_fraction(u)

scale_percent(t, x) = (t, scale_percent(x))
scale_percent(x) = x * 100

# Fig 6: Transmission mitigation

In [ ]:
##### simulation parameters #####

# reduction for adults, fixed to 40%
s = 0.4

beta = copy(β)
beta[2,1] *= (1-s)
beta[2,2] *= (1-s)

# account for increased school contacts
multiplier = 10.0

# reduction for children
reduction = [0.0, 0.2, 0.4, 0.6, 0.8]

In [ ]:
##### figure options #####
common_xticks = 0:14:tspan[end]
common_xtickl = string.(Int.(common_xticks .÷ 7))
rcParams["axes.grid"] = false
rcParams["axes.facecolor"] = "0.90"

#
# ===== one cohort =====
#
# R₀ approx. ? under parameter choices whenever children are at home
# R₀ approx. ? when children attend schools (effect of multiplier)
#
nclasses = 2
ncohorts = 1

fig, ax = subplots(figsize=[2*4.0, 1.5*4.0], nrows=2, ncols=2)

# inset1 = ax[1].inset_axes([0.125, 0.5, 0.3, 0.4])
# inset2 = ax[3].inset_axes([0.125, 0.5, 0.3, 0.4])
#
# inset1.grid(axis="y")
# inset2.grid(axis="y")

for r in reduction
    beta[1,1] = β[1,1] * (1-r)
    beta[1,2] = β[1,2] * (1-r)
    prob = make_model_instance(nclasses, ncohorts, tspan, beta, params,
        mult=multiplier,
        demographics=demographics,
        total_infected=total_infected,
        infected=infected,
    )
    q = dropdims(sum(prob.u0, dims=(1,3)), dims=(1,3))
    sol = solve(prob, saveat=1.0)

    # top left panel
    children = scale_percent(cumulative_child_fraction.(sol.u)) / q[1]
    ax[1].plot(sol.t, children, label="$(100*r)%")

    ymax = 50

    ax[1].set_title("Children (1 cohort)")
    ax[1].set_xticks(common_xticks)
    ax[1].set_xticklabels(common_xtickl)
    # ax[1].set_ylabel("Cumulative cases (%)")
    ax[1].set_yticks(0:10:ymax)
    ax[1].set_ylim(0, ymax)

    # inset
    # inset1.plot(sol.t, children, label="$(100*r)%")
    # inset1.set_xlim(sol.t[1], sol.t[end])
    # inset1.set_ylim(0, 2.0)
    # inset1.set_xticks(common_xticks)
    # inset1.set_yticks(0.0:0.5:2.0)
    # inset1.set_xticklabels("")

    # top right panel
    adults = scale_percent(cumulative_adult_fraction.(sol.u)) / q[2]
    ax[3].plot(sol.t, adults)
    ax[3].set_title("Adults (1 cohort)")
    ax[3].set_xticks(common_xticks)
    ax[3].set_xticklabels(common_xtickl)
    # ax[3].set_ylabel("Cumulative cases (%)")
    ax[3].set_yticks(0:10:ymax)
    ax[3].set_ylim(0, ymax)

    # inset2.plot(sol.t, adults, label="$(100*r)%")
    # inset2.set_xlim(sol.t[1], sol.t[end])
    # inset2.set_ylim(0, 2.0)
    # inset2.set_xticks(common_xticks)
    # inset2.set_yticks(0.0:0.5:2.0)
    # inset2.set_xticklabels("")

    println("1 cohort, $(r*100)% reduction")
    index = findfirst(isequal(4*7), sol.t)
    println("1 month:  $(children[index]), $(adults[index])")
    index = findfirst(isequal(3*4*7), sol.t)
    println("3 months: $(children[index]), $(adults[index])")
    index = findfirst(isequal(6*4*7), sol.t)
    println("6 months: $(children[index]), $(adults[index])")
    println()
end

# ax[1].indicate_inset_zoom(inset1)
# ax[3].indicate_inset_zoom(inset2)

#
# ===== two cohorts =====
#
# R₀ approx. ? under parameter choices whenever children are at home
# R₀ approx. ? when children attend schools (effect of multiplier)
#
nclasses = 2
ncohorts = 2

for r in reduction
    beta[1,1] = β[1,1] * (1-r)
    beta[1,2] = β[1,2] * (1-r)
    prob = make_model_instance(nclasses, ncohorts, tspan, beta, params,
        mult=multiplier,
        demographics=demographics,
        total_infected=total_infected,
        infected=infected,
    )
    q = dropdims(sum(prob.u0, dims=(1,3)), dims=(1,3))
    sol = solve(prob, saveat=1.0)

    # bottom left panel
    children = scale_percent(cumulative_child_fraction.(sol.u)) / q[1]
    ax[2].plot(sol.t, children)

    ymax = 7.0

    ax[2].set_title("Children (2 cohorts)")
    ax[2].set_xlabel("Weeks after reopening")
    ax[2].set_xticks(common_xticks)
    ax[2].set_xticklabels(common_xtickl)
    ax[2].set_ylabel("Cumulative cases (%)")
    ax[2].set_yticks(0:1:ymax)
    ax[2].set_ylim(0, ymax)

    # bottom right panel
    adults = scale_percent(cumulative_adult_fraction.(sol.u)) / q[2]
    ax[4].plot(sol.t, adults)

    ax[4].set_title("Adults (2 cohorts)")
    ax[4].set_xlabel("Weeks after reopening")
    ax[4].set_xticks(common_xticks)
    ax[4].set_xticklabels(common_xtickl)
    ax[4].set_ylabel("Cumulative cases (%)")
    ax[4].set_yticks(0:1:ymax)
    ax[4].set_ylim(0, ymax)

    println("2 cohorts, $(r*100)% reduction")
    index = findfirst(isequal(4*7), sol.t)
    println("1 month:  $(children[index]), $(adults[index])")
    index = findfirst(isequal(3*4*7), sol.t)
    println("3 months: $(children[index]), $(adults[index])")
    index = findfirst(isequal(6*4*7), sol.t)
    println("6 months: $(children[index]), $(adults[index])")
    println()
end

# add legend
reduction_labels = [string(Int(r*100), "%") for r in reduction]

fig.legend(
    reduction_labels,
    title=latexstring("Reduction in transmission; ", L"\beta_{11}, \beta_{12}"),
    bbox_to_anchor=(0.17,0,1.02,0),
    loc="lower left",
    # borderaxespad=-0.2,
    frameon=false,
    ncol=5)
fig.tight_layout()
fig.subplots_adjust(bottom=0.175)

# add labels to each panel
reindex = [1,3,2,4]
for i in eachindex(ax)
    ax[i].text(-0.0, 1.15, string('A'+(reindex[i]-1)), transform=ax[i].transAxes,
      fontsize=16, fontweight="bold", va="top", ha="right")
    ax[i].grid(axis="y")
end

PyPlot.savefig("../figures/Fig6.png", dpi=300, bbox_inches="tight")
PyPlot.savefig("../figures/Fig6.tiff", dpi=300, bbox_inches="tight")

fig

# Appendix

In [ ]:
PyPlot.close_figs()

In [ ]:
using InteractiveUtils; versioninfo()

In [ ]:
using Pkg; Pkg.status()